# Setup

Install dependencies and check PyTorch and GPU.

In [ ]:
print("setting up dependencies......")
!pip install ultralytics --quiet
print("**** setup complete ****")

# 1. Inference

With yolo setup, we'll do a test inference on some images in our data set. We'll be using the `yolov8n.pt` model to see how well it identifies airplanes and helicopters. You should see mostly failures to detect objects and bad classifications like 'boat' and 'kite'.

In [ ]:
%cd /opt/app-root/src/
!yolo predict model=yolov8n.pt source=datasets/training/valid/images exist_ok=True

## Sample Images
Let's see how well the default model did with some random selections. You can run this cell multiple times.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Set the directory path where the image files are located
directory = 'runs/detect/predict'

# Get all image files in the directory
image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Filter image files with "HELI" in the title and without
heli_files = [file for file in image_files if 'HELI' in file]
non_heli_files = [file for file in image_files if 'HELI' not in file]

# Select two random files with "HELI" in the title
selected_heli_files = random.sample(heli_files, 2)

# Select two random files without "HELI" in the title
selected_non_heli_files = random.sample(non_heli_files, 2)

# Create subplots for displaying the images
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Display the selected images with "HELI" in the title
for i, file in enumerate(selected_heli_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[i // 2, i % 2]
    ax.imshow(img)
    ax.axis('off')

# Display the selected images without "HELI" in the title
for i, file in enumerate(selected_non_heli_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[(i+2) // 2, (i+2) % 2]
    ax.imshow(img)
    ax.axis('off')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

# 2. Train
Now we'll train a new model with our flying things dataset. The following will be used to kick it off:

```shell
yolo train model=yolov8n.pt        # use the default yolov8 nano model as a baseline
           batch=2                 # batch size determines the amount of concurrent processing
           epochs=100              # the number of times through the entire dataset
           data=flyingthings.yaml  # file containing the class id and names and the directories for test, train, and valid
           exist_ok=True           # allow results to be overwritten
```

In [ ]:
%cd /opt/app-root/src/
!yolo train model=yolov8n.pt batch=2 epochs=1 data=flyingthings.yaml exist_ok=True

## Results
Once training is complete, we can look at the results. Here are some graphs indicating accuracy over the the course of epoch runs.

In [ ]:
from PIL import Image
import IPython.display as display

# Set the paths to the image files
image_paths = [
    'runs/detect/train/results.png',
    'runs/detect/train/F1_curve.png',
    'runs/detect/train/P_curve.png',
    'runs/detect/train/R_curve.png'
]

# Create a list to hold the Image objects
images = []

# Open the images and append them to the list
for path in image_paths:
    image = Image.open(path)
    images.append(image)

# Display the images
for image in images:
    display.display(image)

# 3 Inference with new model
Now we'll run the same images we used earlier with our new model to see how much better it does at identifying airplanes and helicopters.

In [ ]:
%cd /opt/app-root/src/
!yolo predict model=runs/detect/train/weights/best.pt source=datasets/training/valid/images exist_ok=True

## Sample Images
Here are some random images from inferencing with our new model. You can run this cell multiple times.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Set the directory path where the image files are located
directory = 'runs/detect/predict'

# Get all image files in the directory
image_files = [file for file in os.listdir(directory) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Filter image files with "HELI" in the title and without
heli_files = [file for file in image_files if 'HELI' in file]
non_heli_files = [file for file in image_files if 'HELI' not in file]

# Select two random files with "HELI" in the title
selected_heli_files = random.sample(heli_files, 2)

# Select two random files without "HELI" in the title
selected_non_heli_files = random.sample(non_heli_files, 2)

# Create subplots for displaying the images
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Display the selected images with "HELI" in the title
for i, file in enumerate(selected_heli_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[i // 2, i % 2]
    ax.imshow(img)
    ax.axis('off')

# Display the selected images without "HELI" in the title
for i, file in enumerate(selected_non_heli_files):
    image_path = os.path.join(directory, file)
    img = mpimg.imread(image_path)
    ax = axes[(i+2) // 2, (i+2) % 2]
    ax.imshow(img)
    ax.axis('off')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()

# 4. Save and tag our new model
Now that we have a model that's better at predicting helis and planes, we will save it to our bucket. As we improve our model with additional training and other adjustments, they will be saved and versioned allowing us to use previous versions for comparrisons or other analysis. The tag indicates the build number.

In [ ]:
import os
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from minio import Minio
from minio.commonconfig import Tags

# Disable SSL verification warning
urllib3.disable_warnings(InsecureRequestWarning)

# MinIO server endpoint and access credentials
endpoint_str = os.environ.get('AWS_S3_ENDPOINT')
minio_endpoint = endpoint_str.lstrip('https://')
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

# MinIO bucket information
bucket_name = 'flyingthings'
file_path = 'runs/detect/train/weights/best.pt'
object_name = 'flyingthings.pt'
tag_key = 'build'


# Initialize MinIO client with SSL verification disabled
client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=True, http_client=urllib3.PoolManager(cert_reqs='CERT_NONE'))
file_exists = False
try:
    # Get object information.
    result = client.stat_object(bucket_name, object_name)
    print(
        "last-modified: {0}, size: {1}".format(
            result.last_modified, result.size,
        ),
    )
    file_exists = True
except:
    file_exists = False
    
# Upload the file and set tags
if not file_exists:
    client.fput_object(bucket_name, object_name, file_path)
    tags = Tags(for_object=True)
    tags[tag_key] = '0.0'
    client.set_object_tags(bucket_name, object_name, tags)
    print(f"File '{object_name}' uploaded to bucket '{bucket_name}' and tagged with '{tag_key}: 0.0'.")
else:
    # Retrieve current tag value for the object
    tags = client.get_object_tags(bucket_name, object_name)
    if tags is None:
        current_value = 0.0
    else:
        current_value = float(tags.get(tag_key, '0.0'))
        
    # Upload the new file
    client.fput_object(bucket_name, object_name, file_path)

    # Increment tag value by 0.1
    new_value = current_value + 0.1

    # Set updated tag value for the new file
    tags = Tags(for_object=True)
    tags[tag_key] = str(new_value)
    client.set_object_tags(bucket_name, object_name, tags)
    print(f"File '{object_name}' uploaded to bucket '{bucket_name}'.")
    print(f"Tag '{tag_key}' value updated to '{new_value}'.")
    

# 5. Promote our new model
If we decide this model is good enough to promote to the next stage let's save it with a naming convention that will be picked up buy our automation.

In [ ]:
import os
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from minio import Minio
from minio.commonconfig import Tags

# Disable SSL verification warning
urllib3.disable_warnings(InsecureRequestWarning)

# MinIO server endpoint and access credentials
endpoint_str = os.environ.get('AWS_S3_ENDPOINT')
minio_endpoint = endpoint_str.lstrip('https://')
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

# MinIO bucket information
bucket_name = 'flyingthings'
file_path = 'runs/detect/train/weights/best.pt'
object_name = 'model_custom.pt'
tag_key = 'build'


# Initialize MinIO client with SSL verification disabled
client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=True, http_client=urllib3.PoolManager(cert_reqs='CERT_NONE'))
file_exists = False
try:
    # Get object information.
    result = client.stat_object(bucket_name, object_name)
    print(
        "last-modified: {0}, size: {1}".format(
            result.last_modified, result.size,
        ),
    )
    file_exists = True
except:
    file_exists = False
    
# Upload the file and set tags
if not file_exists:
    client.fput_object(bucket_name, object_name, file_path)
    tags = Tags(for_object=True)
    tags[tag_key] = '0.0'
    client.set_object_tags(bucket_name, object_name, tags)
    print(f"File '{object_name}' uploaded to bucket '{bucket_name}' and tagged with '{tag_key}: 0.0'.")
else:
    # Retrieve current tag value for the object
    tags = client.get_object_tags(bucket_name, object_name)
    if tags is None:
        current_value = 0.0
    else:
        current_value = float(tags.get(tag_key, '0.0'))
        
    # Upload the new file
    client.fput_object(bucket_name, object_name, file_path)

    # Increment tag value by 0.1
    new_value = current_value + 0.1

    # Set updated tag value for the new file
    tags = Tags(for_object=True)
    tags[tag_key] = str(new_value)
    client.set_object_tags(bucket_name, object_name, tags)
    print(f"File '{object_name}' uploaded to bucket '{bucket_name}'.")
    print(f"Tag '{tag_key}' value updated to '{new_value}'.")